# Neural Networks: Theory and Implementation

Group Members:

Name: `Sayeh Jarollahi` \
Student ID (matriculation number): `XXX` \
Email: `sajaXXXX@stud.uni-saarland.de` 

Name: `Mahsa Amani` \
Student ID (matriculation number): `7064006` \
Email: `maam00002@stud.uni-saarland.de`

## Exercise 1

## Exercise 2

## Exercise 3